# Skript1: Finetuning eines BERT-Modells für NER anhand von GermEval2014
Dieses Notebook beschreibt das Training eines Transformer Modells für die Task des Named Entity Recognition. Es wird ein BERT-Modell mit der nativen Hugging Face Transformers Bibliothek trainiert. Das Training erfolgt gemäß des Splits wie es die Shared Task 2014 vorgab.

In [1]:
# Installation der benötigten Bibliotheken
#!pip install transformers
#!pip install datasets

Zunächst werden die Skriptparameter gesetzt. Die `task` Variable gibt an, für welche Aufgabe das Modell trainiert werden soll.
`ner` steht hier für Named Entity Recognition, dabei handelt es sich um ein Token-Klassifizierungsproblem. Die Variable `model_checkpoint` beinhaltet den Namen des zu nutzenden vortrainierten Transformer Modells. Das Modell `deepset/gbert-base` ist ein BERT-Modell, welches mithilfe von deutschen Texten trainiert wurde. Der Name kann durch einen beliebigen Modellcheckpoint aus dem Transformers Model Hub ersetzt werden:  https://huggingface.co/models

In [2]:
task = "ner"
model_checkpoint = 'deepset/gbert-base'
batch_size=32

## Herunterladen des Datasets germeval2014

Neben der Transformers Bibliothek bietet die Hugging Face Inc. mit der `Datasets` Bibliothek eine Sammlung von Datensätzen und Metriken zum herunterladen an. Um diese zu nutzen, werden zuerst die beiden Methoden `load_dataset` und `load_metric` importiert.

In [3]:
from datasets import load_dataset, load_metric

Ebenso wie es für die vortrainierten Modelle einen Hub gibt, gibt es einen Hub für die verfügbaren Datasets: https://huggingface.co/datasets/germeval_14. Mit der Methode `load_dataset` kann das Dataset anschließend heruntergeladen werden.

In [4]:
datasets = load_dataset("germeval_14")

Reusing dataset germ_eval14 (/home/cbartels/.cache/huggingface/datasets/germ_eval14/germeval_14/2.0.0/2a7a0c62dc3278203778c3a16bfbe257d5656aa0f4ad1e84f357f4caa904e0da)


#### Exploration des Aufbaus der Daten

Das heruntergeladene Dataset ist ein `DatasetDcit-Objekt`, welches die Keys `train, validation` und `test` besitzt. Der Value für jeden Key ist das `Dataset-Objekt`, welches die jeweiligen Daten für den Split enthält.

Im Falle des GermEval2014 Datasets bestehen die Daten aus insgesamt 31300 Datensätzen mit den Features `id, source, tokens, ner_tags` und `nested_ner_tags`. Die `nested_ner_tags` finden im Rahmen dieses Anwendungsbeispiels keine Verwendung.

In [5]:
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'source', 'tokens', 'ner_tags', 'nested_ner_tags'],
        num_rows: 24000
    })
    validation: Dataset({
        features: ['id', 'source', 'tokens', 'ner_tags', 'nested_ner_tags'],
        num_rows: 2200
    })
    test: Dataset({
        features: ['id', 'source', 'tokens', 'ner_tags', 'nested_ner_tags'],
        num_rows: 5100
    })
})

Möchte man auf ein einzelnes Item aus dem Datensatz zugreifen, so wählt man den Key des Splits aus wählt anschließend den Index des gewünschten Items aus:

In [6]:
print(datasets["train"][0])

{'id': '0', 'ner_tags': [19, 0, 0, 0, 7, 0, 0, 0, 0, 19, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'nested_ner_tags': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'source': 'n-tv.de vom 26.02.2005 [2005-02-26] ', 'tokens': ['Schartau', 'sagte', 'dem', '"', 'Tagesspiegel', '"', 'vom', 'Freitag', ',', 'Fischer', 'sei', '"', 'in', 'einer', 'Weise', 'aufgetreten', ',', 'die', 'alles', 'andere', 'als', 'überzeugend', 'war', '"', '.']}


Die NER Tags sind bereits Integer ID codiert: 

In [7]:
datasets["train"][0]['ner_tags']

[19, 0, 0, 0, 7, 0, 0, 0, 0, 19, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

Das zugehörige Textlabel findet man unter dem `features`-Attribut des Datasets. Das Dataset GermEval2014 besitzt insgesamt 25 verschiedene Klassen:

In [8]:
datasets["train"].features["ner_tags"]

Sequence(feature=ClassLabel(num_classes=25, names=['O', 'B-LOC', 'I-LOC', 'B-LOCderiv', 'I-LOCderiv', 'B-LOCpart', 'I-LOCpart', 'B-ORG', 'I-ORG', 'B-ORGderiv', 'I-ORGderiv', 'B-ORGpart', 'I-ORGpart', 'B-OTH', 'I-OTH', 'B-OTHderiv', 'I-OTHderiv', 'B-OTHpart', 'I-OTHpart', 'B-PER', 'I-PER', 'B-PERderiv', 'I-PERderiv', 'B-PERpart', 'I-PERpart'], names_file=None, id=None), length=-1, id=None)

Die Klassenliste der Label kann wie folgt extrahiert werden:

In [9]:
label_list = datasets["train"].features[f"{task}_tags"].feature.names
label_list

['O',
 'B-LOC',
 'I-LOC',
 'B-LOCderiv',
 'I-LOCderiv',
 'B-LOCpart',
 'I-LOCpart',
 'B-ORG',
 'I-ORG',
 'B-ORGderiv',
 'I-ORGderiv',
 'B-ORGpart',
 'I-ORGpart',
 'B-OTH',
 'I-OTH',
 'B-OTHderiv',
 'I-OTHderiv',
 'B-OTHpart',
 'I-OTHpart',
 'B-PER',
 'I-PER',
 'B-PERderiv',
 'I-PERderiv',
 'B-PERpart',
 'I-PERpart']

#### Visualisierung von zufälligen Beispieldaten aus dem Dataset 
Die folgende Funktion wählt zufällig `N` Items aus dem übergebenen Dataset aus und gibt diese in einem `Pandas DataFrame` aus. Dabei werden die `ner_tags` in ihre entsprechenden Textlabel dekodiert.

In [10]:
# Quelle in Anlehnung an: https://colab.research.google.com/github/huggingface/notebooks/blob/master/examples/text_classification.ipynb#scrollTo=X6HrpprwIrIz
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10,seed=None):
    assert num_examples <= len(dataset),
    picks = []
    random.seed(seed)
    #Befüllen mit Random indexes
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            random.seed(seed)
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
       # print(column)
       # print(typ)
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
            #die zeile hier unten ist aktiv
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

Die Methode kann nun dazu genutzt werden, eine tabellarische Ausgabe von Beispieldaten aus dem Dataset zu erzeugen. Dafür übergibt man das `datasets` Objekt und gibt den gewünschten Split an:

In [11]:
show_random_elements(datasets["train"], seed = 43)

,id,ner_tags,nested_ner_tags,source,tokens
0,1263,"[O, O, O, O, O, O, B-LOC, O, O]","[O, O, O, O, O, O, O, O, O]",http://de.wikipedia.org/wiki/Khangchenne [2009-12-25],"[Er, hielt, sich, lieber, im, fernen, Ngari, auf, .]"
1,9374,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]",http://www.verbaende.com/News.php4?m=56415 [2008-09-25],"[Daher, lässt, er, keine, Gelegenheit, aus, ,, die, Politik, wachzurütteln, und, zum, Handeln, zu, bewegen, .]"
2,22813,"[B-PER, O, O, O, O, O, O, O, O, B-LOC, O]","[O, O, O, O, O, O, O, O, O, O, O]",http://www.hellwegeranzeiger.de/afp/journal/doc/zuma-anc.htm [2007-12-16],"[Zuma, ist, der, Liebling, der, Armen, und, Benachteiligten, in, Südafrika, .]"
3,4716,"[O, O, O, O, O, O, O, O, B-ORGpart, O, O, O, O, O, O, O]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]",http://www.handelsblatt.com/unternehmen/handel-dienstleister/lufthansa-fuerchtet-deutlichen-gewinnrueckgang;2251398 [2009-04-24],"[Außer, in, der, Cargo-Sparte, stehen, auch, an, dezentralen, Lufthansa-Standorten, des, Passagiergeschäfts, die, Zeichen, auf, Kurzarbeit, .]"
4,15156,"[O, O, O, O, O, B-LOCderiv, O, B-PER, I-PER, I-PER, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]",http://de.wikipedia.org/wiki/Albert_Riera [2009-10-14],"[Nationalmannschaft, Natürlich, entging, auch, dem, spanischen, Nationaltrainer, Luis, Aragonés, Rieras, Leistungssteigerung, nicht, ,, so, dass, er, mittlerweile, drei, Spiele, für, die, "", Selección, "", bestritten, hat, .]"
5,12120,"[O, O, B-ORG, O, O, O, B-LOCderiv, O, O, O]","[O, O, O, O, O, O, O, O, O, O]",welt.de vom 15.02.2005 [2005-02-15],"[Damit, stattet, Walther, künftig, auch, die, irakischen, Sicherheitskräfte, aus, .]"
6,22008,"[O, O, O, O, O, O, O, O, B-PER, I-PER, O, O, O, O]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O]",http://de.wikipedia.org/wiki/Zapp_(Magazin) [2009-11-28],"[Vom, 9., März, 2003, bis, November, 2005, führte, Caren, Miosga, durch, die, Sendung, .]"
7,22886,"[O, O, O, O, O, O, O, O, O, O, B-LOCderiv, O, O, O, O]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]",http://www.handelsblatt.com/unternehmen/banken-versicherungen/ec-karten-debakel-banken-ueben-kritik-an-zka;2527576 [2010-02-11],"[Dem, Konsensprinzip, werde, "", in, Extenso, über, die, Säulen, des, deutschen, Bankensystems, gehuldigt, "", .]"
8,3155,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-OTH, I-OTH, O, O, B-OTH, I-OTH, I-OTH, I-OTH, I-OTH, I-OTH, O, B-PER, I-PER, I-PER, O]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]",http://de.wikipedia.org/wiki/Grosses_vollständiges_Universal-Lexicon_Aller_Wissenschafften_und_Künste [2009-12-29],"[Schon, der, erste, Artikel, zum, Buchstaben, „, A, “, sei, eine, Kompilation, aus, dem, Historischen, Lexicon, und, dem, Allgemeinen, Lexicon, der, Künste, und, Wissenschaften, von, Johann, Theodor, Jablonski, .]"
9,14850,"[B-PER, O, O, O, O, O, O, O, B-ORG, I-ORG, I-ORG, O]","[O, O, O, O, O, O, O, O, O, O, O, O]",http://de.wikipedia.org/wiki/Nanaimo [2009-11-26],"[Dunsmuir, arbeitete, für, diese, Gesellschaft, und, für, die, Harewood, Coal, Company, .]"


## Preprocessing der Daten

Für das Preprocessign der Daten wird der Tokenizer des genutzen Modells benötigt. Dieser kann durch den folgenden Methodenaufruf heruntergeladen werden. <br>
`AutoTokenizer.from_pretrained()` lädt den zum Modell passenden Tokenizer sowie das Vokabular, welches für das Pretraining des Modelcheckpoints genutzt wurde, herunter.

In [12]:
from transformers import  AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast = True)

Der Tokenizer kann direkt genutzt werden, um eine Inputsequenz zu tokenisieren. Dabei erhält man ein Dictionary mit Mappings zu den input_ids, token_type_ids und eine attention_mask.
Die input_id ist die Identifikation des jeweiligen Tokens im Vokabluar des Modells.
Token_type_ids markieren Tokens in Seq2Seq Tasks und geben dem Modell Informationen darüber, zu welchem Teil einer zweiteiligen Eingabesequenz ein Token gehört.
Die attention_mask teilt dem Modell mit, für welche Token die Attention berechnet werden soll. Ist eine Eingabesequenz z. B. sehr kurz im Gegensatz zu den anderen, dann wird diese per Padding auf die gleiche Länge gebracht. Die attention_mask verhindert anschließend, dass die Attention für diese Padding Token berechnet wird.

Übergibt man dem Tokenizer nun eine String Sequenz, erhält man die oben beschriebene Ausgabe zurück:

In [13]:
tokenizer("Hallo, das ist ein Satz!")

{'input_ids': [102, 4485, 818, 199, 215, 143, 5607, 3330, 103], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

Wenn im Datensatz, wie hier gegeben (Germeval_2014) die Inputs schon in einzelne Wörter gesplittet sind, dann kann man diese Liste und den Parameter `is_split_into_words=True` an den Tokenizer übergeben:

In [14]:
tokenizer(["Hallo",",","das", "ist", "ein","Satz","der","in","Wörter","aufgeteilt","wurde","!"],is_split_into_words=True)

{'input_ids': [102, 4485, 818, 199, 215, 143, 5607, 125, 153, 14253, 14070, 325, 3330, 103], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

#### Alignment- Problematik

Transformer Tokenizer nutzen meistens Subword Tokenizer, daher kann es vorkommen, dass selbst diese Wortliste noch in weitere Token zerlegt wird:

In [15]:
example = datasets["train"][5]
print(example["tokens"])

['ARD-Programmchef', 'Günter', 'Struve', 'war', 'wegen', 'eines', 'vierwöchigen', 'Urlaubs', 'für', 'eine', 'Stellungnahme', 'nicht', 'erreichbar', '.']


In [16]:
tokenized_input = tokenizer(example["tokens"],is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

['[CLS]', 'ARD', '-', 'Programm', '##chef', 'Günter', 'Stru', '##ve', 'war', 'wegen', 'eines', 'vier', '##wöch', '##igen', 'Urlaubs', 'für', 'eine', 'Stellungnahme', 'nicht', 'erreichbar', '.', '[SEP]']


Hier wurde zum Beispiel "Struve" in 2 Subtoken zerlegt und "vierwöchigen" sogar in 3 Subtoken.

Durch das Einfügen von SpecialToken [CLS] und [SEP] sowie die Subword Tokenisierung ist die Liste der Token länger als die Liste der zugehörigen Tags. Das Allignment aus dem Dataset ist somit kaputt:

In [17]:
len(example[f"{task}_tags"]),len(tokenized_input['input_ids'])

(14, 22)

Um dieses Problem zu lösen und das Alignment wiederherzustellen besitzt das Rückgabeobjektdes Tokenizer die `word_ids()` Methode.
- Sie liefert eine Liste die genauso lang ist, wie die Liste mit den Input-IDs
- Sie mapped Special Token zu `None` und alle anderen zum zugehörigen Original Wort-Input, bspw. markiert eine `0` die Zugehörigkeit des Subtokens zum ersten Token der Eingabesequenz

In [18]:
tokenized_input.word_ids()

[None, 0, 0, 0, 0, 1, 2, 2, 3, 4, 5, 6, 6, 6, 7, 8, 9, 10, 11, 12, 13, None]

Nun kann man das Alignment zwischen den `ner_tags` und den `input_ids` wiederherstellen. Im Ergbnis haben die Label und die Input_ids die gleiche Anzahl.

In [19]:
word_ids = tokenized_input.word_ids()
aligned_labels = [-100 if i is None else example[f"{task}_tags"][i] for i in word_ids]
print(len(aligned_labels), len(tokenized_input["input_ids"]))

22 22


Mit der obigen Funktion wurden die `ner_tags` für die eingefügten Special Token  auf -100 gestellt und somit von Pytorch ignoriert. Die anderen `input_ids` haben das entsprechene Label ihres zugehörigen Wortes erhalten.

Eine andere Strategie ist es nur das Label für den ersten Token eines Wortes zu setzen und -100 für die weiteren Subtoken des Wortes zu vergeben. Dafür muss folgendes Flag geädnert werden:

In [20]:
label_all_tokens = True

#### Preprocessing Funktion

Mit der folgenden Funktion werden die übergebenen Datensätze des Datasets zunächst tokenisiert und anschließend werden die `ner_tags` mit den tokenisierten `input_ids` alligned. Dafür wird ein `labels` Attribut angelegt, welches diese Zuordnung enthält. Als Rückgabe erhält man die um das `labels` Attribut erweiterte Ausgabe des Tokenizers.

In [21]:
# Quelle https://colab.research.google.com/github/huggingface/notebooks/blob/master/examples/token_classification.ipynb#scrollTo=n9qywopnIrJH
def tokenize_and_align_labels(examples):
    # Tokenisierung der Eingabetoken
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    labels = []
    
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special Tokens haben den Wert None als word id. Durch das Setzen des Wertes -100 als Label
            # wird dieser Token automatisch in der Loss Funktion ignoriert.
            if word_idx is None:
                label_ids.append(-100)
            # Setzen des Labels für den ersten Token eines Wortes.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # Abhängig vom label_all_tokems_flag wird für den nächsten Token eines Wortes das gleiche Label oder -100
            # gesetzt
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

Die Funktion kann mit einem oder mehreren Examples genutzt werden. Wenn mehrere Datensätze übergeben werden, dann gibt der interne Tokenizer der Funktion eine  Liste mit Listen für jeden Key zurück:

In [22]:
tokenize_and_align_labels(datasets['train'][:3])

{'input_ids': [[102, 20652, 976, 30890, 1382, 249, 224, 3424, 9733, 224, 507, 2476, 818, 4086, 745, 224, 153, 369, 3485, 24523, 818, 128, 1446, 1301, 276, 11052, 30888, 285, 224, 566, 103], [102, 5213, 23835, 4022, 2605, 3697, 30881, 3736, 2201, 125, 24168, 7273, 744, 276, 13015, 6974, 818, 276, 180, 397, 11738, 169, 13095, 260, 249, 21799, 7374, 566, 103], [102, 1776, 307, 1413, 333, 249, 13681, 5650, 339, 1910, 566, 1483, 153, 1270, 387, 143, 20175, 1965, 6110, 4496, 371, 818, 215, 2149, 205, 26058, 21494, 566, 103]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

Um die Funktion auf alle Zeilen im Datensatz anzuwenden kann man die `map` Methode des `datasets` Objekts nutzen. Dies wendet die Funktion auf alle Splits im `dataset` an, (training, valid, test set werden alle mit der einen Zeile Code preprocessed).
 - Durch `batched=True` wird dem Tokenizer ermöglicht mehrere Datensätze parallel zu verarbeiten

In [23]:
tokenized_dataset = datasets.map(tokenize_and_align_labels,batched=True)

Loading cached processed dataset at /home/cbartels/.cache/huggingface/datasets/germ_eval14/germeval_14/2.0.0/2a7a0c62dc3278203778c3a16bfbe257d5656aa0f4ad1e84f357f4caa904e0da/cache-3dcba97207658466.arrow
Loading cached processed dataset at /home/cbartels/.cache/huggingface/datasets/germ_eval14/germeval_14/2.0.0/2a7a0c62dc3278203778c3a16bfbe257d5656aa0f4ad1e84f357f4caa904e0da/cache-3f309d3cba5268fd.arrow
Loading cached processed dataset at /home/cbartels/.cache/huggingface/datasets/germ_eval14/germeval_14/2.0.0/2a7a0c62dc3278203778c3a16bfbe257d5656aa0f4ad1e84f357f4caa904e0da/cache-3f5c41a6444aaa50.arrow


Durch das Preprocessing werden die Ausgaben des Tokenizers als Features des DataSets ergänzt.
In den folgenden Ausgaben sieht man, dass nun die `attention_mask, die input_ids, die labels und die token_type_ids` als Features ergänzt wurden.

In [24]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'id', 'input_ids', 'labels', 'ner_tags', 'nested_ner_tags', 'source', 'token_type_ids', 'tokens'],
        num_rows: 24000
    })
    validation: Dataset({
        features: ['attention_mask', 'id', 'input_ids', 'labels', 'ner_tags', 'nested_ner_tags', 'source', 'token_type_ids', 'tokens'],
        num_rows: 2200
    })
    test: Dataset({
        features: ['attention_mask', 'id', 'input_ids', 'labels', 'ner_tags', 'nested_ner_tags', 'source', 'token_type_ids', 'tokens'],
        num_rows: 5100
    })
})

In [25]:
show_random_elements(tokenized_dataset["train"],seed=43,num_examples=2)

,attention_mask,id,input_ids,labels,ner_tags,nested_ner_tags,source,token_type_ids,tokens
0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",1263,"[102, 279, 5637, 251, 6949, 223, 12495, 106, 196, 1127, 30883, 216, 566, 103]","[-100, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, -100]","[O, O, O, O, O, O, B-LOC, O, O]","[O, O, O, O, O, O, O, O, O]",http://de.wikipedia.org/wiki/Khangchenne [2009-12-25],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[Er, hielt, sich, lieber, im, fernen, Ngari, auf, .]"
1,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",9374,"[102, 6053, 2876, 180, 855, 7169, 260, 818, 128, 2017, 17643, 5335, 30007, 136, 386, 15898, 205, 8452, 566, 103]","[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]",http://www.verbaende.com/News.php4?m=56415 [2008-09-25],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[Daher, lässt, er, keine, Gelegenheit, aus, ,, die, Politik, wachzurütteln, und, zum, Handeln, zu, bewegen, .]"


## Finetuning des Models

Für die Konstruktion des Modells wird ein Label-ID Mapping erzeugt. Dieses Mapping wird anschließend in der Configuration des Modells hinterlegt. Es ermöglicht dem Modell die vorhergesagten Label als Texte auszugeben anstatt ihrer numerischen Kodierung.

In [26]:
id2label = {}
label2id= {}
for i,l in enumerate(label_list):
    id2label[str(i)] = l
    label2id[l]=str(i)
print(id2label)
print(label2id)

{'0': 'O', '1': 'B-LOC', '2': 'I-LOC', '3': 'B-LOCderiv', '4': 'I-LOCderiv', '5': 'B-LOCpart', '6': 'I-LOCpart', '7': 'B-ORG', '8': 'I-ORG', '9': 'B-ORGderiv', '10': 'I-ORGderiv', '11': 'B-ORGpart', '12': 'I-ORGpart', '13': 'B-OTH', '14': 'I-OTH', '15': 'B-OTHderiv', '16': 'I-OTHderiv', '17': 'B-OTHpart', '18': 'I-OTHpart', '19': 'B-PER', '20': 'I-PER', '21': 'B-PERderiv', '22': 'I-PERderiv', '23': 'B-PERpart', '24': 'I-PERpart'}
{'O': '0', 'B-LOC': '1', 'I-LOC': '2', 'B-LOCderiv': '3', 'I-LOCderiv': '4', 'B-LOCpart': '5', 'I-LOCpart': '6', 'B-ORG': '7', 'I-ORG': '8', 'B-ORGderiv': '9', 'I-ORGderiv': '10', 'B-ORGpart': '11', 'I-ORGpart': '12', 'B-OTH': '13', 'I-OTH': '14', 'B-OTHderiv': '15', 'I-OTHderiv': '16', 'B-OTHpart': '17', 'I-OTHpart': '18', 'B-PER': '19', 'I-PER': '20', 'B-PERderiv': '21', 'I-PERderiv': '22', 'B-PERpart': '23', 'I-PERpart': '24'}


Erzeugen und Konfiguration des Modells mit Label-ID Mapping:
 - `AutoModelForTokenClassification.from_pretrained` lädt automatisch das entsprechende Modell herunter und initialisiert einen Token-Klassifizierungskopf am Ende des Modells.
 - Die auftretenden Warnungen geben nur Auskunft darüber, dass der Kopf des Modells ausgetauscht wurde und demzufolge keine trainierten Weights hat

In [27]:
from transformers import  AutoModelForTokenClassification, TrainingArguments, Trainer

In [28]:
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list), id2label=id2label, label2id=label2id)

Some weights of the model checkpoint at deepset/gbert-base were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at deepset/gb

#### Vorbereitung für die Erzeugung des `Trainers`:
Es müssen zunächst die `TrainingArguments` konfiguriert werden, dabei handelt es sich um die Hyperparameter des Trainings:
 - Dazu zählen z.B. die Anzahl der Epoch, die Learning Rate, die Batchsize und der weight_decay
 - `load_best_model_at_end` und `metric_for_best_model` sorgen dafür, dass am Ende des Trainings das Model mit der höchsten `F1_Score` geladen wird

In [29]:
metric_name = 'eval_f1'
args = TrainingArguments(
    f"test-{task}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
)

Der `DataCollator` fügt die Beispieldaten zu Batches zusammen und fügt das nötige Padding für die Inputs und Labels ein. (Hierfür wird die Länge des längsten Datensatzes gewählt)

In [30]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

Damit das Modell während des Trainings die gewünschten Metriken berechnen kann, muss eine Funktion definiert werden die diese Metriken berechnet. Das übernimmt die `compute_metric` Funktion:
- Die Datasets Bibliothek ermöglicht es, Funktionen zur Berechnung von Metriken herunterzuladen.
- `Seqeval` eignet sich gut für Tasks im Bereich der TokenClassification: https://github.com/chakki-works/seqeval

In [31]:
#Installieren von seqeval falls nötig
#!pip install seqeval

Mit `load_metric` kann man eine gewünschte Funktion aus der Datasets Bibliothek herunterladen:

In [32]:
metric = load_metric("seqeval")

#### Beispiel für die Anwendung der Metrik

Um die Metriken zu berechnen, müssen der Funktion zwei Listen an die Parameter `predictions` und ` references` übergeben werden. 

Für dieses Beispiel wird die Labelliste für den `predictions` Parameter durch ein Mapping der `ner_tags` auf die Indizies der `label_list` erstellt:

In [33]:
labels = [label_list[i] for i in example[f"{task}_tags"]]

In [34]:
labels

['B-ORGpart',
 'B-PER',
 'I-PER',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O']

Für den `references` Parameter wird die Labelsliste leicht angepasst:

In [35]:
labels_ref = labels.copy()
labels_ref[-1] = 'B-ORGpart'

In [36]:
labels_ref

['B-ORGpart',
 'B-PER',
 'I-PER',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-ORGpart']

Anschließend kann die `compute` Methode des Metrikobjekts benutzt werden, um die Metrik zu berechnen:

In [37]:
metric.compute(predictions=[labels], references=[labels_ref])

{'ORGpart': {'precision': 1.0,
  'recall': 0.5,
  'f1': 0.6666666666666666,
  'number': 2},
 'PER': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 0.6666666666666666,
 'overall_f1': 0.8,
 'overall_accuracy': 0.9285714285714286}

Damit diese Metrik vom Modell berechnet werden kann, muss sie in eine Funktion gewrappt werden.
Diese Funktion erhält während des Trainings das Ergebnis des Methodenaufrufs `Trainer.evaluate`. Dabei handelt es sich um ein Tuple aus den `predictions` und den tatsächlichen `labels`. Die `predictions` sind der letzte Hidden State des Modells, daher muss hiervon der maximale Wert ausgewählt werden, um den tatsächlich vorhergesagten Wert zu erhalten. Die Funktion `compute_metrics` kümmert sich um das nötige Postprocessing und gibt die berechneten Metriken zurück.

In [38]:
import numpy as np

def compute_metrics(p):
    # Unpacking des Tuple p
    predictions, labels = p
    # Größten Wert für die jeweilige Prediction auswählen
    predictions = np.argmax(predictions, axis=2)

    # Ignorieren aller Werte mit -100
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

- hier wird die Ausgabe pro Kategorie weggelassen und nur Gesamt precision/recall/f1/accuracy  ausgegeben

Anschließend kann der `Trainer` erzeugt werden. Diesem wird das Modell, die Hyperparameter, die Trainings- und Evaluierungsdatensätze, der Tokenizer, der DataCollator sowie die `compute_metrics` Funktion übergeben.

In [39]:
trainer = Trainer(
    model,
    args,
    train_dataset =tokenized_dataset["train"],
    eval_dataset =tokenized_dataset["validation"],
    data_collator = data_collator,
    tokenizer = tokenizer,
    compute_metrics = compute_metrics
)

Nun kann das Training mit der `train` Methode gestartet werden

In [40]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy,Runtime,Samples Per Second
1,0.234600,0.112509,0.804340,0.854685,0.828749,0.968251,3.262700,674.284000
2,0.083200,0.102504,0.832221,0.866947,0.849229,0.972393,3.266000,673.613000
3,0.054000,0.110168,0.821120,0.869510,0.844622,0.971893,3.290100,668.663000
4,0.036600,0.116814,0.835001,0.870608,0.852433,0.972559,3.339900,658.706000
5,0.027700,0.123111,0.832720,0.870059,0.850980,0.972411,3.289100,668.870000


/home/cbartels/miniconda3/lib/python3.8/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/cbartels/miniconda3/lib/python3.8/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/cbartels/miniconda3/lib/python3.8/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/cbartels/miniconda3/lib/python3.8/site-packages/seqeval/metri

TrainOutput(global_step=3750, training_loss=0.0796404411315918, metrics={'train_runtime': 528.2913, 'train_samples_per_second': 7.098, 'total_flos': 4200290246980608, 'epoch': 5.0})

Mit der evaluate Methode kann man das Model nochmals gegen das hinterlegte Dataset oder ein anderes Dataset validieren.

In [41]:
trainer.evaluate()

/home/cbartels/miniconda3/lib/python3.8/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'eval_loss': 0.11681380122900009,
 'eval_precision': 0.835000877654906,
 'eval_recall': 0.8706076134699854,
 'eval_f1': 0.8524325777260102,
 'eval_accuracy': 0.9725591715976332,
 'eval_runtime': 3.3819,
 'eval_samples_per_second': 650.52,
 'epoch': 5.0}

Evaluieren gegen das ungesehene Testset:

In [42]:
trainer.evaluate(tokenized_dataset["test"])

{'eval_loss': 0.1268075555562973,
 'eval_precision': 0.8387224286504862,
 'eval_recall': 0.846282705807275,
 'eval_f1': 0.8424856065118125,
 'eval_accuracy': 0.9724039443451621,
 'eval_runtime': 7.8381,
 'eval_samples_per_second': 650.668,
 'epoch': 5.0}

Im Anschluss kann man wieder die Precision/recall/f1/accuracy pro Kategorie berechnen indem man die `predict` Methode nutzt. Dazu wird die folgende `evaluate_all_categories` definiert. Sie erhält ein tokenisiertes Dataset als Eingabeparameter. Anschließend wird die Metrik für jede enthaltene Kategorie berechnet.

In [45]:
def evaluate_all_categories(tokenized_dataset):
    predictions, labels, _ = trainer.predict(tokenized_dataset)
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    return metric.compute(predictions=true_predictions, references=true_labels)

In [47]:
evaluate_all_categories(tokenized_dataset["test"])

/home/cbartels/miniconda3/lib/python3.8/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'LOC': {'precision': 0.8857882623705409,
  'recall': 0.9171879654453381,
  'f1': 0.901214693399678,
  'number': 3357},
 'LOCderiv': {'precision': 0.8366383380547686,
  'recall': 0.9287211740041929,
  'f1': 0.8802781917536014,
  'number': 954},
 'LOCpart': {'precision': 0.7843137254901961,
  'recall': 0.6521739130434783,
  'f1': 0.7121661721068249,
  'number': 368},
 'ORG': {'precision': 0.8087071240105541,
  'recall': 0.7869062901155327,
  'f1': 0.7976577748861419,
  'number': 2337},
 'ORGderiv': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 19},
 'ORGpart': {'precision': 0.7512376237623762,
  'recall': 0.8303693570451436,
  'f1': 0.7888239116309291,
  'number': 731},
 'OTH': {'precision': 0.6893280632411067,
  'recall': 0.6855345911949685,
  'f1': 0.6874260938115885,
  'number': 1272},
 'OTHderiv': {'precision': 0.5487804878048781,
  'recall': 0.6,
  'f1': 0.5732484076433121,
  'number': 75},
 'OTHpart': {'precision': 0.4885057471264368,
  'recall': 0.47752808988764045,
  'f

Das trainierte Modell schneidet besonders gut in den Hauptkategorien `LOC` und `PER` ab. Die nächstbesten Werte werden für die beiden anderen Hauptkategorien `ORG` und `OTH` erreicht. Dieses Ergebnis entspricht der Erwartung, da es für diese Kategorien auch die meisten Trainingsdaten gab. Besonders für die Klasse der `deriv` Token werden nicht so hohe F1-Werte erreicht. 

#### Speichern des trainierten Modells

In [48]:
trainer.save_model('models/deepset_finetuned/ner_deepset')

In [49]:
tokenizer.save_pretrained('models/deepset_finetuned/ner_deepset')

('models/deepset_finetuned/ner_deepset/tokenizer_config.json',
 'models/deepset_finetuned/ner_deepset/special_tokens_map.json',
 'models/deepset_finetuned/ner_deepset/vocab.txt',
 'models/deepset_finetuned/ner_deepset/added_tokens.json')

## Beispielhafte Anwendung des Modells

Um ein Transformers Modell einfach anzuwenden, nutzt man die Methode `pipeline`. Diese baut automatisch in Abhängigkeit vom übergebenen Task und Modell eine Pipeline. Die Pipeline kümmert sich um die nötigen Schritte, um aus einer Stringeingabe eine Modellvorhersage zu erzeugen:

In [1]:
from transformers import pipeline

In [2]:
ner = pipeline("ner", model = f'./models/deepset_finetuned/ner_deepset')

Some weights of BertModel were not initialized from the model checkpoint at ./models/deepset_finetuned/ner_deepset and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Anschließend kann dem `ner` Objekt der zu Verarbeitende String übergeben werden um eine Vorhersage der enthaltenden Entitäten zu erhalten:

In [3]:
ner('''In der CDU liegen die Nerven blank. Parteichef Laschet und andere führende Christdemokraten sprechen
    nach den Niederlagen in Mainz und Stuttgart von einem Warnschuss – und attackieren die SPD. ''')

[{'word': 'CDU',
  'score': 0.9935018420219421,
  'entity': 'B-ORG',
  'index': 3,
  'start': 7,
  'end': 10},
 {'word': 'Las',
  'score': 0.9978851675987244,
  'entity': 'B-PER',
  'index': 12,
  'start': 47,
  'end': 50},
 {'word': '##che',
  'score': 0.9973504543304443,
  'entity': 'B-PER',
  'index': 13,
  'start': 50,
  'end': 53},
 {'word': '##t',
  'score': 0.997477114200592,
  'entity': 'B-PER',
  'index': 14,
  'start': 53,
  'end': 54},
 {'word': 'Mainz',
  'score': 0.9973543286323547,
  'entity': 'B-LOC',
  'index': 25,
  'start': 129,
  'end': 134},
 {'word': 'Stuttgart',
  'score': 0.99721360206604,
  'entity': 'B-LOC',
  'index': 27,
  'start': 139,
  'end': 148},
 {'word': 'SPD',
  'score': 0.9950452446937561,
  'entity': 'B-ORG',
  'index': 37,
  'start': 192,
  'end': 195}]

Das Modell konnte für obige Eingabe alle Token korrekt bestimmen.

In [4]:
ner('''Die Europäische Union hat ein Verfahren gegen Großbritannien wegen Verletzung des EU-Austrittsvertrags
    eingeleitet. Dies teilte die EU-Kommission am Montag in Brüssel mit. ''')

[{'word': 'Europäische',
  'score': 0.9842971563339233,
  'entity': 'B-ORG',
  'index': 2,
  'start': 4,
  'end': 15},
 {'word': 'Union',
  'score': 0.9951105713844299,
  'entity': 'I-ORG',
  'index': 3,
  'start': 16,
  'end': 21},
 {'word': 'Großbritannien',
  'score': 0.9957483410835266,
  'entity': 'B-LOC',
  'index': 8,
  'start': 46,
  'end': 60},
 {'word': 'EU',
  'score': 0.9970673322677612,
  'entity': 'B-ORGpart',
  'index': 12,
  'start': 82,
  'end': 84},
 {'word': '-',
  'score': 0.9964889287948608,
  'entity': 'B-ORGpart',
  'index': 13,
  'start': 84,
  'end': 85},
 {'word': 'Aust',
  'score': 0.9951298832893372,
  'entity': 'B-ORGpart',
  'index': 14,
  'start': 85,
  'end': 89},
 {'word': '##ritts',
  'score': 0.99505215883255,
  'entity': 'B-ORGpart',
  'index': 15,
  'start': 89,
  'end': 94},
 {'word': '##vertrags',
  'score': 0.9963422417640686,
  'entity': 'B-ORGpart',
  'index': 16,
  'start': 94,
  'end': 102},
 {'word': 'EU',
  'score': 0.9509938359260559,
  'e

In [5]:
ner('''Das sogenannte Nordirland-Protokoll im Austrittsvertrag sieht vor,
    dass einige Regeln des EU-Binnenmarkts für Nordirland weiter gelten.
    Dies soll Kontrollen an der Landgrenze zum EU-Staat Irland auf der gemeinsamen
    Insel überflüssig machen. Da Waren dennoch kontrolliert werden müssen, um EU-Standards zu wahren,
    wurden die Kontrollen auf Häfen an der Irischen See zwischen Nordirland und dem übrigen
    Großbritannien verschoben. So wurde das Problem zwischen Großbritannien und der
    EU – und insbesondere der europäischen Republik Irland – zu einem innerbritischen Problem.''')

[{'word': 'Nordirland',
  'score': 0.5609582662582397,
  'entity': 'B-LOCpart',
  'index': 3,
  'start': 15,
  'end': 25},
 {'word': '-',
  'score': 0.6679670214653015,
  'entity': 'B-LOCpart',
  'index': 4,
  'start': 25,
  'end': 26},
 {'word': 'Protokoll',
  'score': 0.611362099647522,
  'entity': 'B-LOCpart',
  'index': 5,
  'start': 26,
  'end': 35},
 {'word': 'EU',
  'score': 0.9962669610977173,
  'entity': 'B-ORGpart',
  'index': 17,
  'start': 94,
  'end': 96},
 {'word': '-',
  'score': 0.9957838654518127,
  'entity': 'B-ORGpart',
  'index': 18,
  'start': 96,
  'end': 97},
 {'word': 'Binnenmarkt',
  'score': 0.9958084225654602,
  'entity': 'B-ORGpart',
  'index': 19,
  'start': 97,
  'end': 108},
 {'word': '##s',
  'score': 0.9950313568115234,
  'entity': 'B-ORGpart',
  'index': 20,
  'start': 108,
  'end': 109},
 {'word': 'Nordirland',
  'score': 0.9927368760108948,
  'entity': 'B-LOC',
  'index': 22,
  'start': 114,
  'end': 124},
 {'word': 'EU',
  'score': 0.994515120983123

Mit den folgenden Eingaben soll getestet werden, wie das Modell auf Eingaben reagiert, die von anderer Natur als die Daten des Trainingsset sind. 

In [14]:
print(ner('''@_A_K_K_ @CDU @jensspahn @_FriedrichMerz Die CDU muss für den packt und die betrügerische
    Wahl karrenbauer muss absteigen auf 12% Überall sind die Regierungen gegen den packt Deutschland
    wird von irren in den Abgrund gerissen.'''))

[{'word': 'CDU', 'score': 0.9770746827125549, 'entity': 'B-ORG', 'index': 10, 'start': 10, 'end': 13}, {'word': 'jens', 'score': 0.8852584958076477, 'entity': 'B-PER', 'index': 12, 'start': 15, 'end': 19}, {'word': '##sp', 'score': 0.8233962059020996, 'entity': 'B-PER', 'index': 13, 'start': 19, 'end': 21}, {'word': '##ahn', 'score': 0.7829808592796326, 'entity': 'B-PER', 'index': 14, 'start': 21, 'end': 24}, {'word': '@', 'score': 0.40987399220466614, 'entity': 'B-PER', 'index': 15, 'start': 25, 'end': 26}, {'word': '_', 'score': 0.6902896761894226, 'entity': 'B-PER', 'index': 16, 'start': 26, 'end': 27}, {'word': 'Friedrich', 'score': 0.8302753567695618, 'entity': 'B-PER', 'index': 17, 'start': 27, 'end': 36}, {'word': '##Mer', 'score': 0.6555641889572144, 'entity': 'B-PER', 'index': 18, 'start': 36, 'end': 39}, {'word': '##z', 'score': 0.4977077841758728, 'entity': 'B-PER', 'index': 19, 'start': 39, 'end': 40}, {'word': 'CDU', 'score': 0.9672029614448547, 'entity': 'B-ORG', 'index':

Das Modell erkennt anhand der obigen Eingabe auch Personenentitäten in Tweets wie bspw. @jensspahn und @_FriedrichMerz

In [15]:
print(ner('@Portugal @Angela_Merkel @SPD @Europäische_Union"'))

[{'word': 'Portugal', 'score': 0.9915956854820251, 'entity': 'B-LOC', 'index': 2, 'start': 1, 'end': 9}, {'word': 'Angela', 'score': 0.9920713901519775, 'entity': 'B-PER', 'index': 4, 'start': 11, 'end': 17}, {'word': 'Merkel', 'score': 0.5476201772689819, 'entity': 'I-PER', 'index': 6, 'start': 18, 'end': 24}, {'word': 'SPD', 'score': 0.989802360534668, 'entity': 'B-ORG', 'index': 8, 'start': 26, 'end': 29}, {'word': 'Europäische', 'score': 0.9796403050422668, 'entity': 'B-ORG', 'index': 10, 'start': 31, 'end': 42}, {'word': 'Union', 'score': 0.7349789142608643, 'entity': 'I-ORG', 'index': 12, 'start': 43, 'end': 48}]


Das Modell hat keine Probleme damit Eigenamen ist Twitter typischen Texten zu extrahieren

In [16]:
print(ner('Portugal Angela_Merkel SPD Europäische_Union"'))

[{'word': 'Portugal', 'score': 0.9774700999259949, 'entity': 'B-LOC', 'index': 1, 'start': 0, 'end': 8}, {'word': 'Angela', 'score': 0.9843271374702454, 'entity': 'B-PER', 'index': 2, 'start': 9, 'end': 15}, {'word': '_', 'score': 0.3647434711456299, 'entity': 'B-PER', 'index': 3, 'start': 15, 'end': 16}, {'word': 'Merkel', 'score': 0.9201464056968689, 'entity': 'B-PER', 'index': 4, 'start': 16, 'end': 22}, {'word': 'SPD', 'score': 0.9769794344902039, 'entity': 'B-ORG', 'index': 5, 'start': 23, 'end': 26}, {'word': 'Europäische', 'score': 0.9590712785720825, 'entity': 'B-ORG', 'index': 6, 'start': 27, 'end': 38}, {'word': '_', 'score': 0.7440067529678345, 'entity': 'B-ORG', 'index': 7, 'start': 38, 'end': 39}, {'word': 'Union', 'score': 0.8858205676078796, 'entity': 'B-ORG', 'index': 8, 'start': 39, 'end': 44}]


In [17]:
print(ner('$Portugal $Angela_Merkel $SPD $Europäische_Union"'))

[{'word': 'Portugal', 'score': 0.9916786551475525, 'entity': 'B-LOC', 'index': 2, 'start': 1, 'end': 9}, {'word': 'Angela', 'score': 0.9911957383155823, 'entity': 'B-PER', 'index': 4, 'start': 11, 'end': 17}, {'word': 'Merkel', 'score': 0.6196739077568054, 'entity': 'I-PER', 'index': 6, 'start': 18, 'end': 24}, {'word': 'SPD', 'score': 0.9880979061126709, 'entity': 'B-ORG', 'index': 8, 'start': 26, 'end': 29}, {'word': 'Europäische', 'score': 0.953593373298645, 'entity': 'B-ORG', 'index': 10, 'start': 31, 'end': 42}, {'word': 'Union', 'score': 0.6619184613227844, 'entity': 'B-ORG', 'index': 12, 'start': 43, 'end': 48}]


Auch bei der Verwendung anderer Sonderzeichen gelingt es dem Modell robuste Vorhersagen zu tätigen.